In [ ]:
## About Data:


In [ ]:
## Importing Main Libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
## Load Data
train_path = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/'

## Image Augmentation:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rotation_range=20,width_shift_range=0.1,height_shift_range=0.1,
rescale=1/225,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,validation_split=0.2)
                               

train=image_gen.flow_from_directory(directory=train_path,target_size=(224,224),classes=['Parasitized','Uninfected'],batch_size=32,shuffle=True,subset="training")
valid=image_gen.flow_from_directory(directory=train_path,target_size=(224,224),classes=['Parasitized','Uninfected'],batch_size=32,shuffle=True,subset="validation")

In [ ]:
## Target Class
train.class_indices
valid.class_indices

In [ ]:
## Train-Batch Visualization:
import matplotlib.pyplot as plt
%matplotlib inline

imgs,label=next(train)
def plotImages(img_arr):
    fig,axes=plt.subplots(1,10,figsize=(20,20))
    axes=axes.flatten()
    for img,ax in zip(img_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
plotImages(imgs)
print(label)

In [ ]:
## Loading VGG-16
vgg16=tf.keras.applications.vgg16.VGG16()
type(vgg16)
vgg16.summary()


In [ ]:
## Customizing VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten

# Creating New Sequential Model(to replicate Vgg16)
model=Sequential(name="Customized_Vgg16")     ## creates new model(without last 3 vgg-16 layer)
for layer in vgg16.layers[:-3]:
    model.add(layer) 

# Creating Dense layers for New-Model
for layer in model.layers:
    layer.trainable=False    # Freeze traing for old layers
#model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=128,activation='relu'))
#model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=2,activation='softmax',name="op"))

model.summary()

In [ ]:
## Model Compile
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


## Early Stpping:
es=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)
## ReduceLROnPlateau:
red_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,min_lr=0.0001)
## CSV Logger:
#csvlogger=tf.keras.callbacks.CSVLogger(filename=r"C:\Desktop\file.txt",separator=',',append=False)


## Model Training
history=model.fit(x=train,steps_per_epoch=len(train),validation_data=valid,epochs=15,verbose=1,callbacks=[es,red_lr])

In [ ]:
import pandas as pd
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()
losses[['accuracy','val_accuracy']].plot()

In [ ]:
predictions = model.predict(valid)
#predictions = predictions>0.5
print('Confusion Matrix: \n',confusion_matrix(valid.classes,predictions),'\n')
print('Classification Report: \n\n',classification_report(valid.classes,predictions))

In [ ]:
model.save('model.h5')